In [15]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans

In [33]:
#reading data from csv files
user_feat = pd.read_csv('user_features.csv')


num = np.arange(500,5000)
num = num[0:len(user_feat)]
user_feat['user_guid'] = num
print user_feat

spice_feat = pd.read_csv('spice_features.csv')

print spice_feat
#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']
#print food_user.head()

    Age Sex     Location  NonVeg  user_guid
0    55   F  South India       0        500
1    54   M      Haryana       2        501
2    59   F      Haryana       1        502
3    60   F         U.P.       2        503
4    37   F  Maharashtra       0        504
5    40   F  Maharashtra       0        505
6    67   F         U.P.       0        506
7    87   F       Punjab       2        507
8    74   F  Maharashtra       1        508
9    26   F          J&K       1        509
10   36   F      Haryana       0        510
11   49   M      Haryana       2        511
12   47   F  South India       1        512
13   98   M  Maharashtra       0        513
14   13   M          J&K       1        514
15    6   F      Haryana       0        515
16   24   M         U.P.       2        516
17   51   F          J&K       2        517
18   45   M  Maharashtra       0        518
19   88   F      Haryana       2        519
20   71   F         U.P.       1        520
21   30   M         U.P.       0

In [17]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})

In [21]:
print "enter the user id"

input_user_id = input()



enter the user id
555


In [22]:
kmeans = KMeans(n_clusters = 10,random_state=0).fit(user_feat)
user_feat['label'] = kmeans.labels_
#print user_feat

index = np.where(user_feat['user_guid']==input_user_id)[0]
#print index[0]
user_label = user_feat.loc[index[0],'label']
#print user_label
user_feat = user_feat.sort_values(['label','Location'],ascending=[1,0])
user = user_feat.drop_duplicates(subset=['Location','label'])

user = user.groupby('label')['Location'].nlargest(3)
print user.head()
loc = user[user_label].tolist()
print loc

label    
0      0     15
       4     10
       3      8
1      70    15
       69    10
Name: Location, dtype: int64
[8, 6, 5]


In [23]:
print "enter veg/non-beg"

veggy = input()

print "enter user required calories of current meal"

upper_calorie = input()

enter veg/non-beg
2
enter user required calories of current meal
200


In [24]:
newspice_feat = spice_feat.set_index('food_id')
spice = newspice_feat[newspice_feat['Location'].isin(loc)]
#print spice
spice = spice[newspice_feat['NonVeg']==veggy]
#print spice
s_spice =  spice.sort_values('count',ascending=[0])
s_index = food_user.index[food_user[input_user_id]==0]
#print s_index

new_spices = s_spice[s_spice.index.isin(s_index)]
print new_spices
new_spices = new_spices[new_spices['calorie']<upper_calorie]
print new_spices

         spice1  spice2  spice3  spice4  count  Location  NonVeg  calorie
food_id                                                                  
60076         1       0       1       1      4         5       2      365
60035         1       1       1       0      3         6       2      423
60015         0       1       1       0      3         5       2      473
60006         1       1       1       1      2         8       2      195
60028         0       0       1       0      2         8       2      120
60091         0       1       1       1      2         6       2      311
60008         0       0       1       1      1         8       2      333
60046         1       1       1       0      1         5       2      434
60048         1       0       0       0      1         8       2      272
60053         1       1       1       0      1         6       2      256
60013         1       1       0       1      1         8       2      187
60011         1       1       0       

/home/demonicode/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [25]:
recommend = []
for i in range(min(3,len(new_spices))):
    recommend.append(new_spices.index[i])
    
if(len(recommend)==0):
    other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
    other_state_spice = other_state_spice.sort('count',ascending= [0])
    new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]

    for i in range(min(3,len(new_other_state))):
        recommend.append(new_other_state.index[i])

    
print recommend    

[60006, 60028, 60013]


In [36]:
spice = np.array(spice_feat.iloc[:,0:4])
#print spice
user_df = pd.DataFrame(np.random.randint(low=-1, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
print user_df.head()
user = np.array(user_df)
#user tells rating and spice tells if ingrediant is there or not.
print user
global spice
global user

user_guid  500  501  502  503  504  505  506  507  508  509 ...   590  591  \
0           -1    1    1    0    0    0    1    1   -1    1 ...     0    1   
1            0   -1   -1   -1   -1   -1    1    0    0    0 ...     0    0   
2           -1   -1   -1    0    1    1   -1    0    1   -1 ...     1   -1   
3            0    1   -1   -1   -1    0   -1   -1    1    0 ...    -1   -1   
4            0   -1    0    0    1    0    0   -1    0    1 ...     0   -1   

user_guid  592  593  594  595  596  597  598  599  
0            1    0    0    1    1    0    0   -1  
1            1    1    0    1   -1    0    1    0  
2           -1    0    0    1    0   -1    0    1  
3           -1   -1    0    0    0   -1    0   -1  
4            0   -1    0   -1    1    0    0   -1  

[5 rows x 100 columns]
[[-1  1  1 ...,  0  0 -1]
 [ 0 -1 -1 ...,  0  1  0]
 [-1 -1 -1 ..., -1  0  1]
 ..., 
 [-1  1 -1 ..., -1  1 -1]
 [ 1  1  1 ..., -1 -1 -1]
 [ 1  1 -1 ...,  1  0  0]]


In [37]:
# we have user_table which will give rating to a particular dish
# content based recommeder system
# we have 2 tables with us

#one is food - spices table and another is food-user rating table
#we will use tf-idf approach
#matrix of spice will contain only of 1 and 0's so need to normalize it

#normalization along the row
def spice_norm():
    
    num_attr = np.sum(spice,axis=1)
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0): 
            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))
        else:
            doc_vec.append(spice[i,:])
    return np.array(doc_vec)      
    
    
    
def attr_norm():
    
    num_attr = np.sum(spice,axis=0)
    
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0):
            doc_vec.append(spice[:,i]*1.0 / num_attr[i])
        else:
            doc_vec.append(spice[:,i])
    return np.array(doc_vec)

def user_v(spice_mat):
    user_vec = []
    
    
    #print user[:,0]
    #print spice_mat[:,0]
    
    for i in range(user.shape[1]):
        user_vec.append([np.dot(user[:,i],spice_mat[:,j]) for j in range(spice_mat.shape[1])])
    
    print (user_vec)
    return np.array(user_vec)
    
def weight_predict(spice_mat):
        
    user_vec = user_v(spice_mat)
    
    preds = []
    #print spice_mat
    for i in range(spice_mat.shape[0]):
        preds.append([np.dot(spice_mat[i,:],user_vec[j,:]) for j in range(user_vec.shape[0])])

    return np.array(preds)
    
    
def idf_weight_pred():
    spice_mat = spice_norm()
    attr_wght = attr_norm()
    #print attr_wght
    weighted_spice = []
    
    for i in range(spice_mat.shape[0]):
        weighted_spice.append([np.dot(spice_mat[i,:],attr_wght[:,j]) for j in range(attr_wght.shape[0])])
        
    preds  = weight_predict(spice_mat = np.array(weighted_spice))     
    
    return np.array(preds)
    


preds = idf_weight_pred()

#print preds.shape

[[0.17334552953201054, 0.03625604263862385, 0.10567195168609342, 0.26547632586042758], [0.16291636224577949, 0.098356975189231546, 0.12116235462415828, 0.15700794288792203], [0.029318140048247628, 0.084942910502825444, -0.0608786761885214, 0.082398665169552507], [-0.084970530362070196, -0.11085270597597038, -0.064731948769622169, -0.091906523108306021], [0.10488126653922465, 0.10168250758203778, 0.0062321028469761244, 0.1888902283113065], [-0.18970242619620153, -0.12075129172669075, -0.13535802360574864, -0.15011368015722726], [0.029608546733252916, 0.0054506877613141602, -0.026751480580557565, 0.00085129565128446577], [-0.060576384541264815, -0.028620374502109483, -0.056113869011094067, -0.095565738470209396], [0.069504158528779392, 0.0061306132823308393, 0.097708385551479246, 0.19088130449568116], [0.1362581984965707, 0.18142453375860718, 0.0630004854819841, 0.18511239127231036], [0.01589322573532926, 0.065891030691499056, 0.053826644650233776, -0.017040981416858562], [-0.11765033577

In [44]:
outputdf = pd.DataFrame(data = preds,index = newspice_feat.index ,columns = user_df.columns)
#print user_df
#print outputdf
s_index = food_user.index[food_user[input_user_id]==0]
#print s_index

output = outputdf[outputdf.index.isin(s_index)]
#print user_df.columns
newoutput = output[input_user_id].sort_values(ascending = False)
#print newoutput
food_to_recommend = list(newoutput[0:3].index)

print food_to_recommend

[60073, 60091, 60077]
